**Model 3: Competitive Pricing**

In [5]:
# ---------------------------------- MODEL 3: COMPETITIVE PRICING ----------------------------------

!pip install pathway bokeh panel --quiet

# -------------------- Imports & Preprocessing --------------------
import pandas as pd
import pathway as pw
import bokeh.plotting
import panel as pn
from math import radians, cos, sin, sqrt, atan2

# Load and clean data
df = pd.read_csv("dataset.csv")
df.columns = df.columns.str.strip()
df = df.rename(columns={"SystemCodeNumber": "LotID"})

# Map traffic to numbers
traffic_map = {"low": 1, "average": 2, "high": 3}
df["TrafficConditionNearby"] = (
    df["TrafficConditionNearby"]
    .astype(str)
    .str.strip()
    .str.lower()
    .map(traffic_map)
    .fillna(1)
)

# Timestamp
df["Timestamp"] = pd.to_datetime(
    df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
    format="%d-%m-%Y %H:%M:%S"
)

# Final selected columns
df = df[[
    "Timestamp", "LotID", "Occupancy", "Capacity", "QueueLength",
    "TrafficConditionNearby", "IsSpecialDay", "VehicleType", "Latitude", "Longitude"
]]
df.to_csv("parking_stream.csv", index=False)

# -------------------- Pathway Schema & Stream --------------------
class ParkingSchema(pw.Schema):
    Timestamp: str
    LotID: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: float
    IsSpecialDay: int
    VehicleType: str
    Latitude: float
    Longitude: float

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    LotID = data.LotID
)

# -------------------- Model 3 Logic --------------------
BASE_PRICE = 10.0
ALPHA = 0.5
BETA = 0.2
GAMMA = 0.1
DELTA = 1.0
EPSILON = 0.8
LAMBDA = 0.6

vehicle_weights = {
    "bike": 0.5,
    "car": 1.0,
    "truck": 1.5
}

# Mock competitor prices based on lat/lon groups (simplified simulation)
@pw.udf
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

@pw.udf
def competitive_price(
    occupancy, capacity, queue, traffic, special_day,
    vehicle_type, latitude, longitude
) -> float:
    if capacity == 0:
        return BASE_PRICE

    occ_ratio = occupancy / capacity
    vehicle_weight = vehicle_weights.get(vehicle_type.lower(), 1.0)

    demand = (
        ALPHA * occ_ratio +
        BETA * queue -
        GAMMA * traffic +
        DELTA * special_day +
        EPSILON * vehicle_weight
    )

    norm_demand = (demand - 0.5) / 3
    norm_demand = max(0, min(norm_demand, 1))
    price = BASE_PRICE * (1 + LAMBDA * norm_demand)

    # Mock logic: if lat > threshold → competitor cheaper, reduce price
    if latitude > 18.6 and price > 12:
        price -= 1.5
    elif latitude < 18.5:
        price += 1.0

    return round(max(5, min(20, price)), 2)

# -------------------- Apply to Pathway Stream --------------------
price_stream = data_with_time.with_columns(
    price = competitive_price(
        data_with_time.Occupancy,
        data_with_time.Capacity,
        data_with_time.QueueLength,
        data_with_time.TrafficConditionNearby,
        data_with_time.IsSpecialDay,
        data_with_time.VehicleType,
        data_with_time.Latitude,
        data_with_time.Longitude
    )
)

# -------------------- Bokeh Visualization --------------------
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400, width=800, title="Model 3: Competitive Pricing",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="darkred")
    fig.scatter("t", "price", source=source, size=6, color="black")
    return fig

viz = price_stream.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [6]:
# -------------------- Run Stream --------------------
import sys
import contextlib
with contextlib.redirect_stdout(sys.stdout):
    pw.run()

Output()